In [16]:
# ******************************************************************************************************************************
# *** ¿ Qué es la REGRESIÓN LOGÍSTICA ?:
# *** Es un MODELO SUPERVISADO que devuelve propabilidades de éxito ( entre 0 y 1 ) haciendo uso de la transformación logarítmica.
# *** La regresión logística es un modelo estadístico que utiliza la función logística, o FUNCION LOGIT, en matemáticas como la
# *** ecuación entre 'X' e 'Y'. La función logit mapea 'Y' como una función sigmoidea de 'X' ( Ver imagen de la función )
# *** Mas informacion en el enlace:
# *** https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# ******************************************************************************************************************************


from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# DataSet PARA PROBAR LA REGRESION LOGÍSTICA ( Modelo Supervisado ):
# HORAS DE ESTUDIO: Horas de estudio diarias de alumnos de matemáticas.
# APROBADO: 'Target' dicotómico sobre si aprobaron o suspendieron la asignatura.

df = pd.DataFrame({
    "Horas_estudio": [0.5, 0.75, 1, 1.25, 1.5, 1.75, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 4, 4.25, 4.5, 4.75, 5, 5.5],
    "Aprobado": [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
})

#df

# Vamos a ENTRENAR NUESTRO MODELO con los DATOS DE ENTRADA ( en este caso serían "datos de ENTRENAMIENTO")
x = np.array(df["Horas_estudio"]).reshape(-1, 1)
y = np.array(df["Aprobado"])
resultado_reg_log = LogisticRegression().fit(x,y)

# La formula de una regresion lineal es: Y = INTERCEPTO + ( PENDIENTE * X )

# Ordenada en el origen:
print(f"INTERCEPTO: Ordenada en el origen del modelo de regresión logística {resultado_reg_log.intercept_.round(2)}")
# Estimación del coeficiente:
print(f"PENDIENTE: Estimadores de las variables independientes del modelo de regresión logística {resultado_reg_log.coef_.round(2)}")

# Nuestra REGRESION LOGISTICA QUEDA CALCULADA COMO: logit(P) = -3.14 + 1.15 * X ( Horas de Estudio )
# Ahora la aplicamos a un nuevo DataSet ( desconocido por el modelo ) que serían los "datos de TEST"

nuevo_vector_horas = np.array([1.5, 2, 3, 2.5, 0.75, 5]).reshape(-1,1)

probabilidades_prediccion = resultado_reg_log.predict_proba(nuevo_vector_horas)
print("Probabilidades estimadas por el modelo de regresión logística: \n", probabilidades_prediccion[:,1].round(3))

# *** Hasta este punto hemos obtenido PROBABILIDADES ( % ), pero queremos conseguir valores concreto ( 1 o 0 - Aprueba o No ).
# *** Para ello se establecen los "PUNTOS DE CORTE" ( que por defecto es 0,5 ). En este caso es perfecto porque lo que pase de
# *** ese 0'5 ( 1+0 / 2 ) sera APROBADO ( y por debajo sera SUSPENSO ) --> Aprobarían 2 alumnos de los 6

prediccion = resultado_reg_log.predict(nuevo_vector_horas)
print("Punto de corte 0.5",prediccion)

# *** NOTA: EL PUNTO DE CORTE se puede modificar a gusto ( y criterio ) del analista:
threshold = 0.4 # Se establece en 0'4
pred = (resultado_reg_log.predict_proba(nuevo_vector_horas) > threshold).astype('int')
print("Punto de corte 0.4",pred[:,1])

INTERCEPTO: Ordenada en el origen del modelo de regresión logística [-3.14]
PENDIENTE: Estimadores de las variables independientes del modelo de regresión logística [[1.15]]
Probabilidades estimadas por el modelo de regresión logística: 
 [0.195 0.301 0.576 0.433 0.093 0.931]
Punto de corte 0.5 [0 0 1 0 0 1]
Punto de corte 0.4 [0 0 1 1 0 1]


In [25]:
# *********************************************************************************************************
# **** 1.- TECNICA TRAIN-TEST VALIDATION
# *** NUNCA se hace el entrenamiento del modelo con el 100% del DataSet --> Provocaria SOBREAJUSTE
# *** Por ello, se divide en dos partes "Train" ( 75-80% ) y "Test" ( 25-20% )
# *** ES UNA DE LAS TECNICAS MAS UTILIZADAS
# *********************************************************************************************************

from sklearn.model_selection import train_test_split
#train_test_split aleatoriamente divide la base de datos en train y validación antendiendo a un porcentaje.

x = np.array(df["Horas_estudio"]).reshape(-1, 1)
y = np.array(df["Aprobado"])
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20)
#print("x_train", x_train)
#print("x_test", x_test)
#print("y_train", y_train)
#print("y_test", y_test)
#x_train: suconjunto de variables independientes asociado a train
#y_train: subconjuto de variable target asociado a train
#x_test: suconjunto de variables independientes asociado a test
#y_test: subconjuto de variable target asociado a test

modelo_ajusta_train = LogisticRegression().fit(x_train, y_train)
#Ordenada en el origen:
print(f"INTERCEPTO: {modelo_ajusta_train.intercept_.round(2)}")
#Estimación del coeficiente:
print(f"PENDIENTE: {modelo_ajusta_train.coef_.round(2)}")

# Nuestra REGRESION LOGISTICA QUEDA CALCULADA COMO: logit(P) = -2.55 + 0.99 * X ( Horas de Estudio )

# SCORE aplica el modelo ajustado en TRAIN en la parte de TEST y devuelve el coeficiente de determinación del modelo de regresión. 
# Esto es, la varianza explicada por lo predicho frente a la variable real, a mayor valor, mejor.
result = modelo_ajusta_train.score(x_test, y_test)
result

INTERCEPTO: [-2.91]
PENDIENTE: [[1.08]]


0.75

In [29]:
# *********************************************************************************************************
# *** 2.- TECNICA K-FOLDS CROSS VALIDATION
# *** Es igual que la anterior pero divide el DataSet en K-Partes ( donde una de ellas testea el modelo cada vez )
# *** Se asegura de utilizar TODOS los registros para entrenar el modelo ( no deja un % aparate --> Evita sesgo )
# *** Es LA MAS UTILIZADA DE TODAS !!! ( es la más robusta de todas, pero también la más lenta )
# *********************************************************************************************************

from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=10)
model = LogisticRegression()

result = cross_val_score(model, x, y, cv=kfold)
result
result.mean()

# Devuelve un array por cada iteracion, dando la puntuacion conseguida. Al final se calcula una media de todos.
# N os quedariamos con el que mejor puntuacion tenga ( con los que tengan 1 )

0.8

In [11]:
# *********************************************************************************************************
# *** 3.- TECNICA LOO CROSS VALIDATION
# *** Divide el DataSet en K-Partes ( donde K es el numero de registros - Ver imagen )
# *** Es iterativo y emplea TODAS las observaciones MENOS UNA como entrenamiento y "esa una" como validacion
# *** NADA RECOMENDABLE !!.. Porque prueba el modelo sobre una unica observacion
# *********************************************************************************************************

from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
model = LogisticRegression()
result_loo = cross_val_score(model, x, y, cv=loo)
result_loo

array([1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 1., 1.])

In [12]:
# ********************************************************************************************************************
# *** 4.- TECNICA SHUFFLE SPLIT CROSS VALIDATION
# *** La division entre "train" y "test" es totalmente aleatoria por lo que las particiones pueden coincidir entre si.
# *** Tampoco es recomendable
# *** Mas informacion en el enlace:
# *** https://scikit-learn.org/stable/modules/cross_validation.html
# ********************************************************************************************************************

from sklearn.model_selection import ShuffleSplit, cross_val_score

kfold = ShuffleSplit(n_splits=10, test_size = 0.20, random_state=2)
model = LogisticRegression()

result = cross_val_score(model, x, y, cv=kfold)
result

array([1.  , 0.5 , 0.75, 1.  , 1.  , 0.75, 0.5 , 0.75, 0.5 , 0.75])